#Отчет об Rush00


In [ ]:
import re
from collections import Counter
from enum import Enum
import urllib
import requests
from bs4 import BeautifulSoup
import ssl
import timeit

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Класс Links

In [1]:
import re
from collections import Counter
from enum import Enum
import urllib
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import ssl

class Links:
	"""
	Analyzing data from links.csv
	"""
	def __init__(self, path_to_the_file):
		"""
		Put here any fields that you think you will need.
		"""
		data = []
		# проверка на открытие файла
		with open(path_to_the_file, 'r') as f:
			str_data = (f.read().lower())
		data = str_data.split('\n')
		next_data = []
		for i in data[1:len(data) - 1]:
			data = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", i)
			# next_data.append(movieId, imdbId, tmdbId)
			next_data.append(data)
		# print(next_data)
		self.data_file = next_data
		self.list_movies_data = []
		for i in range(0, 10):
			ssl._create_default_https_context = ssl._create_unverified_context
			url = f'https://www.imdb.com/title/tt{self.data_file[i][1]}/'
			headers = {'User-Agent': 'My User Agent 1.0',
			'From': f'https://www.imdb.com/title/tt{self.data_file[i][1]}/'
			}
			page = urllib.request.urlopen(urllib.request.Request(url=url, headers=headers)).read()
			soup = BeautifulSoup(page, 'lxml')
			tags = soup.find_all('li', role='presentation', class_='ipc-metadata-list__item')
			for tag in tags:
				if (tag.text.find('Director') != -1):
					Director = tag.text.replace('Director', '')
					break
			for tag in tags:
				if (tag.text.find('Budget') != -1):
					Budget = tag.text.replace('Budget', '')
					break
			for tag in tags:
				if (tag.text.find('Also known as') != -1):
					film = tag.text.replace('Also known as', '')
					break
			for tag in tags:
				if (tag.text.find('Gross worldwide') != -1):
					Gross = tag.text.replace('Gross worldwide', '')
					break
			for tag in tags:
				if (tag.text.find('Runtime') != -1):
					Runtime = tag.text.replace('Runtime', '')
					break
			field_data = [self.data_file[i][1], film, Director, Budget, Gross, Runtime]
			self.list_movies_data.append(field_data)

	def get_imdb(list_of_movies, list_of_fields):
		"""
		The method returns a list of lists [movieId, field1, field2, field3, ...] for the list of movies given as the argument (movieId).
		For example, [movieId, Director, Budget, Cumulative Worldwide Gross, Runtime].
		The values should be parsed from the IMDB webpages of the movies.
		Sort it by movieId descendingly.
		"""
		list_movies_data = []
		for i in list_of_movies:
			ssl._create_default_https_context = ssl._create_unverified_context
			url = f'https://www.imdb.com/title/tt{i}/'
			headers = {'User-Agent': 'My User Agent 1.0',
			'From': f'https://www.imdb.com/title/tt{i}/'
			}
			page = urllib.request.urlopen(urllib.request.Request(url=url, headers=headers)).read()
			soup = BeautifulSoup(page, 'lxml')
			tags = soup.find_all('li', role='presentation', class_='ipc-metadata-list__item')
			field_data = [i]
			for field in list_of_fields:
				for tag in tags:
					if (tag.text.find(field) != -1):
						field_value = tag.text.replace(field, '')
						break
				field_data.append(field_value)
			list_movies_data.append(field_data)
		imdb_info = list(sorted(list_movies_data, key=lambda x: (-int(x[0]))))
		return imdb_info

		
	def top_directors(self, n):
		"""
		The method returns a dict with top-n directors where the keys are directors and 
		the values are numbers of movies created by them. Sort it by numbers descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			list_movies.append([self.list_movies_data[i][2], self.list_movies_data[i][0]])
		a = sorted(list_movies)
		directors = dict()
		for i in a:
			directors[i[0]] = i[1]
		return directors
		
	def most_expensive(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are their budgets. Sort it by budgets descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			list_movies.append([self.list_movies_data[i][1], self.list_movies_data[i][3]])
		a = sorted(list_movies)
		budgets = dict()
		for i in a:
			budgets[i[0]] = i[1]
		return budgets
		
	def most_profitable(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are the difference between cumulative worldwide gross and budget.
		Sort it by the difference descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			gross = ''.join(x for x in self.list_movies_data[i][4] if x.isdigit())
			budget = ''.join(x for x in self.list_movies_data[i][3] if x.isdigit())
			list_movies.append([self.list_movies_data[i][1], int(gross) - int(budget)])
		a = sorted(list_movies, key = lambda x: -x[1])
		profits = dict()
		for i in a:
			profits[i[0]] = i[1]
		return profits
		
	def longest(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are their runtime. If there are more than one version â€“ choose any.
		Sort it by runtime descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			Runtime_min = int(int(self.list_movies_data[i][5].split(' ')[0]) * 60) + int(self.list_movies_data[i][5].split(' ')[2])
			field_data = [self.list_movies_data[i][1], self.list_movies_data[i][5], Runtime_min]
			list_movies.append(field_data)
		a = sorted(list_movies, key = lambda x: -x[2])
		runtimes = dict()
		for i in a:
			runtimes[i[0]] = i[1]
		return runtimes
		
	def top_cost_per_minute(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are the budgets divided by their runtime. The budgets can be in different currencies â€“ do not pay attention to it. 
		The values should be rounded to 2 decimals. Sort it by the division descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			Runtime_min = int(int(self.list_movies_data[i][5].split(' ')[0]) * 60) + int(self.list_movies_data[i][5].split(' ')[2])
			budget = ''.join(x for x in self.list_movies_data[i][3] if x.isdigit())
			field_data = [self.list_movies_data[i][1], round(int(budget) / Runtime_min, 2)]
			list_movies.append(field_data)
		a = sorted(list_movies, key = lambda x: -x[1])
		costs = dict()
		for i in a:
			costs[i[0]] = i[1]
		return costs

Проверка методов класса Links

In [ ]:
link = Links("/content/drive/MyDrive/links.csv")
print(Links.get_imdb(['0114709'], ['Director', 'Budget', 'Gross worldwide', 'Runtime']))
%timeit(Links.get_imdb(['0114709'], ['Director', 'Budget', 'Gross worldwide', 'Runtime']))


TypeError: ignored

In [ ]:
n = 10
print(link.top_directors(n))
%timeit(link.top_directors(n))

In [ ]:
print(link.most_expensive(n))
%timeit(link.most_expensive(n))

In [ ]:
print(link.most_profitable(n))
%timeit(link.most_profitable(n))

In [ ]:
print(link.longest(n))
%timeit(link.longest(n))

In [ ]:
print(link.top_cost_per_minute(n))
%timeit(link.top_cost_per_minute(n))

Класс Tags

In [ ]:
class Tags:
	def __init__(self, path_to_the_file):
		data = []
		# проверка на открытие файла
		with open(path_to_the_file, 'r') as f:
			str_data = (f.read().lower())
		data = str_data.split('\n')
		next_data = []
		for i in data[1:len(data) - 1]:
			userId, movieId, tag, timestamp = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", i)
			next_data.append(tag)
		# print(next_data)
		self.data_teg= next_data
		# tag = Counter(next_data)
		
	
	def most_words(self, n):
		"""
		The method returns top-n tags with most words inside. It is a dict 
 		where the keys are tags and the values are the number of words inside the tag.
		Drop the duplicates. Sort it by numbers descendingly.
        """
		big_tags = dict()
		if (len(self.data_teg) < n or n <= 0):
			raise ValueError('n > count teg or n <= 0')
		else:
			no_sorted_tuple = dict(zip(self.data_teg, (map(lambda x: len(x.split(' ')), self.data_teg))))
			sorted_tuple = dict(sorted(no_sorted_tuple.items(), key=lambda x: -x[1]))
			count = 0
			for key, value in sorted_tuple.items():
				if (count != n):
					big_tags[key] = value
					count +=1
		return big_tags

	def longest(self, n):
		"""
		The method returns top-n longest tags in terms of the number of characters.
		It is a list of the tags. Drop the duplicates. Sort it by numbers descendingly.
		"""
		if (len(self.data_teg) < n or n <= 0):
			raise ValueError('n > count teg or n <= 0')
		else:
			sorted_tuple = dict(zip(self.data_teg, (map(lambda x: len(x), self.data_teg))))
			big_tags = dict(sorted(sorted_tuple.items(), key=lambda x: -x[1]))
			count = 0
			for key, value in sorted_tuple.items():
				if (count != n):
					big_tags[key] = value
					count +=1
		return big_tags

	def most_words_and_longest(self, n):
		"""
		The method returns the intersection between top-n tags with most words inside and 
		top-n longest tags in terms of the number of characters.
		Drop the duplicates. It is a list of the tags.
		"""

		return big_tags
        
	def most_popular(self, n):
		"""
		The method returns the most popular tags. 
		It is a dict where the keys are tags and the values are the counts.
		Drop the duplicates. Sort it by counts descendingly.
		"""
		if (len(self.data_teg) < n or n <= 0):
			raise ValueError('n > count teg or n <= 0')
		else:
			popular_tags = Counter(self.data_teg).most_common(n)
		return popular_tags
        
	def tags_with(self, word):
		"""
		The method returns all unique tags that include the word given as the argument.
		Drop the duplicates. It is a list of the tags. Sort it by tag names alphabetically.
		"""
		tags_with_word = set(filter(lambda x: x.find(word) != -1, self.data_teg))
		return tags_with_word

Проверка методов класса Tags

In [ ]:
tags = Tags("/content/drive/MyDrive/tags.csv")
print(tags.most_words(10))
%timeit(tags.most_words(10))

{'something for everyone in this one... saw it without and plan on seeing it with kids!': 16, 'the catholic church is the most corrupt organization in history': 10, 'villain nonexistent or not needed for good story': 8, '06 oscar nominated best movie - animation': 7, 'it was melodramatic and kind of dumb': 7, 'oscar (best music - original score)': 6, 'stop using useless characters for filler': 6, 'oscar (best effects - visual effects)': 6, 'everything you want is here': 5, 'based on a true story': 5}
1000 loops, best of 5: 1.75 ms per loop


In [ ]:
print(tags.longest(10))
%timeit(tags.most_words(10))

{'something for everyone in this one... saw it without and plan on seeing it with kids!': 85, 'the catholic church is the most corrupt organization in history': 63, 'villain nonexistent or not needed for good story': 48, 'r:disturbing violent content including rape': 43, '06 oscar nominated best movie - animation': 41, 'stop using useless characters for filler': 40, 'academy award (best supporting actress)': 39, 'oscar (best effects - visual effects)': 37, 'audience intelligence underestimated': 36, 'it was melodramatic and kind of dumb': 36, 'r:sustained strong stylized violence': 36, 'oscar (best music - original score)': 35, 'start of a beautiful friendship': 31, 'oscar (best supporting actress)': 31, 'assassin-in-training (scene)': 28, 'oscar (best cinematography)': 27, 'everything you want is here': 27, 'political right versus left': 27, 'avant-garde romantic comedy': 27, 'r:disturbing violent images': 27, 'representation of children': 26, 'not available from netflix': 26, 'rita h

In [ ]:
print(tags.most_words_and_longest(10))
%timeit(tags.most_words(10))

In [ ]:
print(tags.most_popular(10))
%timeit(tags.most_popular(10))

In [ ]:
print(tags.tags_with('a'))
%timeit(tags.tags_with('a'))

Класс Movie

In [ ]:
import re
from collections import Counter
from enum import Enum
from functools import reduce
from typing import List


class Movie:
    def __init__(self, id, title, genres):
        self.id = id
        self.title = title
        try:
            self.year = int(re.sub('.*\(([0-9]{4})\).*', r'\1', title))
        except:
            self.year = 2000
        self.genres = genres.split('|')

class Movies:
    """
    Analyzing data from movies.csv
    """

    def __init__(self, path_to_the_file):
        """
        Put here any fields that you think you will need.
        """
        with open(path_to_the_file) as file:
            self.movies: List[Movie] = []
            next(file)
            for line in file:
                movieId, title, genres = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", line.strip())
                self.movies.append(Movie(movieId, title, genres))

    def dist_by_release(self):
        """
        The method returns a dict or an OrderedDict where the keys are years and the values are counts. 
        You need to extract years from the titles. Sort it by counts descendingly.
        """
        return dict(Counter(map(lambda x: x.year, self.movies)))

    def dist_by_genres(self):
        """
        The method returns a dict where the keys are genres and the values are counts.
    Sort it by counts descendingly.
        """
        return dict(Counter(reduce(lambda a, b: a + b, map(lambda x: x.genres, self.movies))))

    def most_genres(self, n):
        """
        The method returns a dict with top-n movies where the keys are movie titles and 
        the values are the number of genres of the movie. Sort it by numbers descendingly.
        """
        # return movies

In [ ]:
movies = Movies("/content/drive/MyDrive/movies.csv")
print(movies.dist_by_genres())
%timeit(movies.dist_by_genres())

In [ ]:
print(movies.dist_by_release())
%timeit(movies.dist_by_release())

In [ ]:
n = 10
print(movies.most_genres(n))
%timeit(movies.most_genres(n))

Класс Ratings

In [ ]:
class Ratings:
	"""
	Analyzing data from ratings.csv
	"""
	def __init__(self, path_to_the_file):
		"""
		Put here any fields that you think you will need.
		"""
	class Movies:    
		def dist_by_year(self):
			"""
			The method returns a dict where the keys are years and the values are counts. 
			Sort it by years ascendingly. You need to extract years from timestamps.
			"""
			return ratings_by_year
		
		def dist_by_rating(self):
			"""
			The method returns a dict where the keys are ratings and the values are counts.
			Sort it by ratings ascendingly.
			"""
			return ratings_distribution
        
		def top_by_num_of_ratings(self, n):
			"""
			The method returns top-n movies by the number of ratings. 
			It is a dict where the keys are movie titles and the values are numbers.
		Sort it by numbers descendingly.
			"""
			return top_movies
		
		def top_by_ratings(self, n, metric=average):
			"""
			The method returns top-n movies by the average or median of the ratings.
			It is a dict where the keys are movie titles and the values are metric values.
			Sort it by metric descendingly.
			The values should be rounded to 2 decimals.
			"""
			return top_movies
        
		def top_controversial(self, n):
			"""
			The method returns top-n movies by the variance of the ratings.
			It is a dict where the keys are movie titles and the values are the variances.
			Sort it by variance descendingly.
			The values should be rounded to 2 decimals.
			"""
			return top_movies

		class Users:
			"""
			In this class, three methods should work. 
			The 1st returns the distribution of users by the number of ratings made by them.
			The 2nd returns the distribution of users by average or median ratings made by them.
			The 3rd returns top-n users with the biggest variance of their ratings.
			Inherit from the class Movies. Several methods are similar to the methods from it.
			"""

Проверка методов класса Ratings



In [ ]:
rating = Ratings("/content/drive/MyDrive/ratings.csv")
rating_moves = rating.Movies()
print(rating_moves.dist_by_year())
%timeit(rating_moves.dist_by_year())

In [ ]:
print(rating_moves.top_by_num_of_ratings())
%timeit(rating_moves.dist_by_year())

In [ ]:
print(rating_moves.top_by_num_of_ratings())
%timeit(rating_moves.dist_by_rating())

In [ ]:
print(rating_moves.top_by_ratings())
%timeit(rating_moves.top_by_ratings())

In [ ]:
print(rating_moves.top_controversial())
%timeit(rating_moves.top_controversial())

In [ ]:
rating_users = rating.Users()